In [1]:
import numpy as np
import gymnasium

import panda_gym
import pybullet
from sb3_contrib import TQC
import evaluate

In [2]:
# evaluation settings
n_substeps=20
max_ep_steps = 200
goal_condition="halt"
num_episodes = 100
#register envs
panda_gym.register_reach_ao(max_ep_steps)

# visual stuff
sphere_list = []
sphere_count = 0
human = True # render the environment during evaluation

In [3]:
# default path options
default_path = "../training/run_data/wandb"
default_model_location = 'files/best_model.zip'
default_yaml_location = "files/config.yaml"

# model names
model_names = ["learning_test_blind"]

# model paths
model_paths = [f"{default_path}/{model_name}/{default_model_location}" for model_name in model_names]

In [4]:
from classes.train_config import TrainConfig
import yaml

with open(f"{default_path}/{model_names[0]}/{default_yaml_location}", 'r') as stream:
    try:
        yaml_config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
        
configuration = TrainConfig()
# omit wandb version
for key, value in yaml_config.items():
    if isinstance(value, dict):
        setattr(configuration, key, value["value"])

# configuration.n_substeps = n_substeps
# configuration.goal_condition = goal_condition

configuration

TrainConfig(name='learning_test_blind', job_type='train', algorithm='TQC', replay_buffer_class='stable_baselines3.her.her_replay_buffer.HerReplayBuffer', policy_type='MultiInputPolicy', learning_starts=10000, prior_steps=0, seed=0, n_envs=8, env_name='PandaReachAO-v3', randomize_robot_pose=False, truncate_on_collision=True, terminate_on_success=True, fixed_target=None, reward_type='sparse', collision_reward=-100, goal_condition='reach', ee_error_thresholds=[0.05, 0.05, 0.05], speed_thresholds=[0.5, 0.1, 0.01], max_timesteps=600000, max_ep_steps=[75, 150, 200], n_substeps=20, stages=['base1', 'base2', 'wangexp_3'], success_thresholds=[0.8, 0.8, 1.0], eval_freq=2000, obs_type=['ee', 'js'], control_type='js', action_limiter='clip', limiter='sim', task_observations={'obstacles': 'vectors', 'prior': None}, render=False, show_goal_space=False, show_debug_labels=False, hyperparams='<classes.hyperparameters.Hyperparameters object at 0x00000244487631D0>')

In [5]:
# default scenario settings
prior_orientation="fkine"
show_goal_space = True
cameraTargetPosition = (0.06289377063512802, 0.05091303586959839, 0.27599984407424927)
cameraDistance = 1.359999656677246
cameraPitch = -33.20000076293945
cameraYaw = 51.800025939941406

# training env
evaluation_scenario="wangexp_3"
prior_orientation="fkine"
cameraTargetPosition = (0.06289377063512802, 0.05091303586959839, 0.27599984407424927)
cameraDistance = 1.359999656677246
cameraPitch = -33.20000076293945
cameraYaw = 51.800025939941406

# narrow_tunnel
evaluation_scenario="narrow_tunnel"
prior_orientation="left"
cameraTargetPosition = (0.6126745939254761, -0.002699121832847595, 0.3287460505962372)
cameraDistance = 1.119999885559082
cameraPitch = -42.800045013427734
cameraYaw = 221.39991760253906

# library 1
evaluation_scenario="library1"
prior_orientation="back"
cameraTargetPosition = (0.09789270907640457, 0.09457920491695404, 0.3600000739097595)
cameraDistance = 1.5399994850158691
cameraPitch = -36.4000129699707
cameraYaw = 49.800018310546875

# library 2
evaluation_scenario="library2"
prior_orientation="back"
cameraTargetPosition = (0.09789270907640457, 0.09457920491695404, 0.3600000739097595)
cameraDistance = 1.5399994850158691
cameraPitch = -36.4000129699707
cameraYaw = 49.800018310546875

# workshop
evaluation_scenario="workshop"
prior_orientation="fkine"
cameraTargetPosition = (-0.05694245547056198, -0.08760415762662888, 0.3400000035762787)
cameraDistance = 1.299999713897705
cameraPitch = -42.8000373840332
cameraYaw = 224.19996643066406

# industrial
evaluation_scenario="industrial"

In [6]:
# bookshelves
evaluation_scenario="bookshelves"

# kitchen
evaluation_scenario="kitchen"

# tunnel
evaluation_scenario="tunnel"

# raised_shelves
evaluation_scenario="raised_shelves"


# tabletop
evaluation_scenario="tabletop"

# tabletop2
evaluation_scenario="tabletop2"

# warehouse
evaluation_scenario="warehouse"

# get config parameters

In [7]:
ee_error_threshold = configuration.ee_error_thresholds[-1]
speed_threshold = configuration.speed_thresholds[-1]

## init work environment

In [8]:
# visual options
configuration.show_debug_labels = False
configuration.show_goal_space = False

# get env
work_env = gymnasium.make(configuration.env_name,
    render=True,
    config=configuration,
                          scenario=evaluation_scenario,
                          ee_error_threshold=ee_error_threshold,
                            speed_threshold=speed_threshold)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ladmin\\PycharmProjects\\panda-gym\\panda_gym\\assets\\scenarios\\bookshelves\\bookshelves.json'

In [9]:
work_env.close()

NameError: name 'work_env' is not defined

## Prior Benchmark

# collect prior results
controller_config = {
            'name': 'custom1',
            "velocity_gain": 0.5,  # gain for the velocity control
            "threshold_error": 0.05,  # tolerance of the final error between the robot's pose and desired pose
            "gain_control_minimization": 0.01,  # gain for the control minimization
            "min_angle_joint": 0.05,  # minimum angle in which the joint is allowed to approach to its limit
            "min_angle_joint_damp_active": 0.9,  # influence angle to joints in which the velocity damper becomes active
            'min_dist_obstacle': 0.1,  # minimum distance in which the link is allowed to approach the object shape
            'min_dist_obstacle_damp_active': 0.3,
            # influence distance to obstacles in which the damper becomes active
            'damp_gain': 1.0,  # The gain for the velocity damper
        }

work_env.unwrapped.robot.control_type = "js"
results, metrics = evaluate.perform_benchmark("prior", work_env.unwrapped, human=False, num_episodes=num_episodes, deterministic=True,
                                     strategy=controller_config,
                                     scenario_name=evaluation_scenario,
                                     prior_orientation=prior_orientation)
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]

evaluation_results = {evaluation_scenario: {"results": results, "metrics": metrics}}

evaluate.display_and_save_benchmark_results(agent_type='prior', eval_type='manual', evaluation_results=evaluation_results, strategy=controller_config)

## agent benchmark

In [12]:
# collect agent results
work_env.unwrapped.robot.control_type = "js"
    
ensemble = []
for model_name in model_names:
    ensemble.append(TQC.load(fr"../training/run_data/wandb/{model_name}/files/best_model.zip", env=work_env,
                     custom_objects={"action_space": gymnasium.spaces.Box(-1.0, 1.0, shape=(7,),dtype=np.float32)}))  # for some reason it won't read action space sometimes)

results, metrics = evaluate.perform_benchmark(ensemble, work_env, human=False, num_episodes=num_episodes, deterministic=True,
                                     strategy="variance_only",
                                     scenario_name=evaluation_scenario,
                                     prior_orientation="")
ee_pos = metrics["end_effector_positions"]
path_success = metrics["path_success"]
done_events  = metrics["done_events"]
work_env.close()

results

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


narrow_tunnel: 100%|██████████| 100/100 [00:31<00:00,  3.20it/s]


{'mean_reward': -108.2079,
 'success_rate': 0.15,
 'collision_rate': 0.85,
 'timeout_rate': 0.0,
 'num_episodes': 100,
 'mean_ep_length': 24.44,
 'mean_ep_length_success': 16.9333,
 'mean_num_sim_steps': 488.8,
 'mean_num_sim_steps_success': 338.6667,
 'mean_effort': 0.8335,
 'mean_manipulability': 0.1756,
 'mean_norm_jerk': 1.545,
 'mean_ee_speed': 0.8951,
 'model_0_action_rate': 24.44}

pybullet.setRealTimeSimulation(0)
loggingID = pybullet.startStateLogging(loggingType=pybullet.STATE_LOGGING_VIDEO_MP4, fileName = f"videos/{evaluation_scenario}_mt-cl-ensemble.mp4", physicsClientId=1)

## Visualize results

In [6]:
# visual options
configuration['show_debug_labels'] = True
configuration['show_goal_space'] = True

# get rendered env
env = gymnasium.make(configuration.env_name,
    render=True,
    config=configuration,
                          scenario=evaluation_scenario,
                          ee_error_threshold=ee_error_threshold,
                            speed_threshold=speed_threshold)

pybullet.resetDebugVisualizerCamera(cameraDistance=cameraDistance, cameraYaw=cameraYaw, cameraPitch=cameraPitch, cameraTargetPosition=cameraTargetPosition, physicsClientId=env.sim.physics_client._client)

C:\Users\ladmin\PycharmProjects\panda-gym\venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(


# visualize/play prior results
env.unwrapped.robot.control_type = "js"
#pybullet.setRealTimeSimulation(enableRealTimeSimulation=1, physicsClientId=1)

_, _ = evaluate.perform_benchmark("prior", env, human=True, num_episodes=num_episodes, deterministic=True,
                                     strategy=controller_config,
                                     scenario_name=evaluation_scenario,
                                     prior_orientation=prior_orientation,
                                  pre_calc_metrics=metrics)


pybullet.stopStateLogging(loggingID)
env.close()

env.close()

In [10]:

_, _ = evaluate.perform_benchmark(ensemble, env, human=True, num_episodes=num_episodes, deterministic=True,
                                     strategy='variance_only',
                                     scenario_name=evaluation_scenario,
                                     prior_orientation=prior_orientation,
                                  pre_calc_metrics=metrics)

C:\Users\ladmin\PycharmProjects\panda-gym\venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.task to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.task` for environment variables or `env.get_wrapper_attr('task')` that will search the reminding wrappers.
  logger.warn(
wangexp_3:   0%|          | 0/100 [00:00<?, ?it/s]C:\Users\ladmin\PycharmProjects\panda-gym\venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.sim to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sim` for environment variables or `env.get_wrapper_attr('sim')` that will search the reminding wrappers.
  logger.warn(
C:\Users\ladmin\PycharmProjects\panda-gym\venv\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.robot to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you c

error: Not connected to physics server.

# get rendered env
env = gymnasium.make(configuration["env_name"],
                     render=True, control_type="js",
                     obs_type=configuration["obs_type"], goal_distance_threshold=0.05,
                     goal_condition="halt",
                     reward_type=reward_type, limiter=configuration["limiter"],
                     show_goal_space=False, scenario=evaluation_scenario,
                     randomize_robot_pose=False,  # if evaluation_scenario != "wang_3" else True,
                     task_observations="vectors+all",
                     truncate_on_collision=True,
                     terminate_on_success=True,
                     show_debug_labels=True, n_substeps=n_substeps)

In [8]:

# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=1, rgbaColor=[1,1,1,0.5], physicsClientId=1)
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=3, rgbaColor=[1,1,1,1], physicsClientId=1)
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=2, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=1, rgbaColor=[1,1,1,0.5])
# pybullet.changeVisualShape(objectUniqueId=0, linkIndex=0, rgbaColor=[1,1,1,0.5])

# todo: draw a circle
time.sleep(3)
radius = 0.9
theta = 0.49*np.pi
phi = 2*np.pi


def get_next_point(radius, phi, theta):
    return [radius * np.sin(theta) * np.cos(phi), radius * np.sin(theta) * np.sin(phi), radius * np.cos(theta)]

inc = -phi/4
xyz1 = get_next_point(radius,inc,theta)
th= 0.0
while th < theta:
    while inc < phi/4:
        inc += 0.01
        xyz2 = get_next_point(radius, inc, theta)
        pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([0,1,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
        time.sleep(0.01)
        xyz1 = xyz2
    th += 0.01

# inc = -0.5*np.pi
# xyz1=get_next_point(phi, theta*inc)
# while inc < 0.5*np.pi:
#     inc += 0.01
#     xyz2 = get_next_point(phi, theta*inc)
#     pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([0,1,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
#     time.sleep(0.01)
#     xyz1 = xyz2

inc = -0.5*np.pi
xyz1=get_next_point(radius, phi/2, theta*inc)
while inc < 0.5*np.pi:
    inc += 0.01
    xyz2 = get_next_point(radius, phi/4, theta*inc)
    pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([0,1,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
    time.sleep(0.01)
    xyz1 = xyz2

rad = -0.9
xyz1=[0.0, rad, 0.01]
while rad < radius:
    rad += 0.01
    xyz2 = [0.0, rad, 0.01]
    pybullet.addUserDebugLine(lineFromXYZ=xyz1, lineToXYZ=xyz2, lineColorRGB=np.array([0,1,0]), physicsClientId=1, lifeTime=0, lineWidth=2)
    time.sleep(0.01)
    xyz1 = xyz2


time.sleep(3)
env.task.create_goal_outline()
while True:
    time.sleep(1)
    env.reset()

In [9]:
#env.close()